In [ ]:
pip install sktime scikit-learn numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.5/141.5 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [ ]:
import numpy as np
import pandas as pd
from sktime.transformations.panel.rocket import Rocket
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer


In [ ]:
import pandas as pd
import numpy as np


file_path = "trainDataset.csv"
trainDataset = pd.read_csv(file_path, index_col=[0, 1])



In [ ]:
trainDataset

Open        High         Low       Close   Adj Close  \
Instance Time                                                               
0        0     118.540001  118.989998  116.900002  117.169998  109.322571   
         1     116.970001  117.360001  115.209999  116.629997  108.818733   
         2     116.690002  116.949997  115.620003  115.970001  108.202942   
         3     115.540001  115.540001  113.220001  113.279999  105.693108   
         4     112.739998  113.930000  111.550003  113.330002  105.739761   
...                   ...         ...         ...         ...         ...   
2236     22     23.200001   23.350000   22.629999   22.820000   21.308558   
         23     23.129999   24.059999   23.020000   23.840000   22.261002   
         24     23.690001   24.490000   23.690001   24.440001   22.821260   
         25     24.360001   26.059999   24.360001   26.000000   24.277939   
         26     26.160000   27.540001   26.150000   27.450001   25.631899   

                  Volume  Pattern  
Instance Time                      
0        0      191900.0        1  
         1      289700.0        1  
         2      389400.0        1  
         3      321100.0        1  
         4      284700.0        1  
...                  ...      ...  
2236     22     525100.0        4  
         23     275700.0        4  
         24     358200.0        4  
         25     793800.0        4  
         26    1210500.0        4  

[58507 rows x 7 columns]

In [ ]:
testDataset

NameError: name 'testDataset' is not defined

In [ ]:
import pandas as pd
import numpy as np


file_path = "testDataset.csv"
testDataset = pd.read_csv(file_path, index_col=[0, 1])

In [ ]:
testDataset

Open        High         Low       Close   Adj Close  \
Instance Time                                                               
0        0     233.874573  238.951889  233.152924  237.757736  229.645874   
         1     237.070450  239.518906  233.728516  234.037796  226.052811   
         2     232.903778  236.554977  230.249146  230.807556  222.932800   
         3     230.060135  232.044678  229.192444  231.039520  223.156845   
         4     232.036087  232.886597  229.475952  231.134018  223.248123   
...                   ...         ...         ...         ...         ...   
561      4      10.610000   10.790000   10.480000   10.640000    9.700566   
         5      10.560000   10.830000   10.560000   10.690000    9.746150   
         6      10.600000   10.760000   10.490000   10.720000    9.773503   
         7      10.550000   10.650000   10.350000   10.550000    9.618512   
         8      10.780000   10.810000   10.480000   10.600000    9.664098   

                  Volume  Pattern  
Instance Time                      
0        0      888481.0        0  
         1      782906.0        0  
         2      548011.0        0  
         3      547196.0        0  
         4     1675694.0        0  
...                  ...      ...  
561      4      331800.0        4  
         5      334000.0        4  
         6      430700.0        4  
         7      306200.0        4  
         8      440200.0        4  

[14267 rows x 7 columns]

In [ ]:
import numpy as np

def prepare_rocket_data(dataset, features, target='Pattern'):
    """
    Prepares data for ROCKET classifier.

    Args:
    - dataset (DataFrame): MultiIndex DataFrame with levels ['Instance', 'Time'].
    - features (list): List of feature columns (e.g., ['Open', 'High', ...]).
    - target (str): Target column name (default: 'Pattern').

    Returns:
    - X (np.array): Features reshaped to (num_samples, series_length, num_features).
    - y (np.array): Target labels (one per instance).
    """
    # Determine the maximum series length
    series_length = dataset.index.get_level_values('Time').max() + 1
    num_features = len(features)
    num_samples = len(dataset.index.get_level_values('Instance').unique())

    # Prepare feature matrix (X)
    X = np.full((num_samples, series_length, num_features), fill_value=np.nan)  # Initialize with NaNs
    for instance, group in dataset.groupby(level=0):  # Group by 'Instance'
        time_steps = group.index.get_level_values('Time').values
        X[instance, time_steps, :] = group[features].values  # Fill in feature values

    # Prepare target labels (y)
    y = dataset.groupby(level=0)[target].first().values  # Take the first target for each instance

    return X, y

# Define features
features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

# Prepare training and testing data
X_train, y_train = prepare_rocket_data(trainDataset, features)
X_test, y_test = prepare_rocket_data(testDataset, features)

print(f"X_train shape: {X_train.shape}")  # Expect (num_samples, series_length, num_features)
print(f"y_train shape: {y_train.shape}")  # Expect (num_samples,)


X_train shape: (2237, 275, 6)
y_train shape: (2237,)


In [ ]:
print(X_train[0])  # Features for the first sample


[[1.18540001e+02 1.18989998e+02 1.16900002e+02 1.17169998e+02
  1.09322571e+02 1.91900000e+05]
 [1.16970001e+02 1.17360001e+02 1.15209999e+02 1.16629997e+02
  1.08818733e+02 2.89700000e+05]
 [1.16690002e+02 1.16949997e+02 1.15620003e+02 1.15970001e+02
  1.08202942e+02 3.89400000e+05]
 ...
 [           nan            nan            nan            nan
             nan            nan]
 [           nan            nan            nan            nan
             nan            nan]
 [           nan            nan            nan            nan
             nan            nan]]


In [ ]:

print(y_train[0])  # Target for the first sample


1


In [ ]:
X_train.shape

(2237, 275, 6)

In [ ]:
import numpy as np
import pandas as pd

def adjust_series_length(group, target_length):
    """
    Adjusts a time series to a target length.
    - Truncates if longer than target_length.
    - Pads with zeros if shorter than target_length.

    Args:
    - group (DataFrame): Subgroup of data (single instance).
    - target_length (int): Desired series length.

    Returns:
    - np.array: Adjusted series as a NumPy array.
    """
    series = group.values
    current_length = len(series)

    if current_length > target_length:
        return series[:target_length]
    else:
        # Padding with zeros if shorter
        padding = np.zeros((target_length - current_length, series.shape[1]))
        return np.vstack([series, padding])


In [ ]:
# Define features and target
features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
target = 'Pattern'
series_length = 275  # Target series length for ROCKET

# === Adjust Series Length ===
def prepare_rocket_data(dataset, features, target, series_length):
    # Group by 'Instance' and adjust each series length
    adjusted = dataset.groupby(level=0).apply(
        lambda group: adjust_series_length(group[features], series_length)
    )

    # Stack adjusted arrays
    X = np.stack(adjusted.values)  # Shape: (num_samples, series_length, num_features)

    # Extract targets (one per instance)
    y = dataset.groupby(level=0)[target].first().values  # Shape: (num_samples,)

    return X, y

# Prepare training and testing data
X_train, y_train = prepare_rocket_data(trainDataset, features, target, series_length)
X_test, y_test = prepare_rocket_data(testDataset, features, target, series_length)

print(f"X_train shape: {X_train.shape}")  # Expect (2237, 275, 6)
print(f"y_train shape: {y_train.shape}")  # Expect (2237,)


X_train shape: (2237, 275, 6)
y_train shape: (2237,)


In [ ]:
X_train = np.transpose(X_train, (0, 2, 1))
X_test = np.transpose(X_test, (0, 2, 1))

In [ ]:
print(y_train)

[1 1 0 ... 0 0 4]


In [ ]:
y_test

array([0, 4, 1, 6, 1, 1, 1, 4, 5, 4, 2, 0, 0, 4, 3, 2, 1, 1, 4, 1, 3, 6,
       0, 6, 5, 1, 6, 0, 5, 2, 3, 3, 4, 0, 4, 6, 1, 5, 3, 4, 2, 2, 5, 6,
       4, 5, 1, 4, 0, 4, 5, 1, 0, 5, 4, 5, 3, 4, 4, 2, 1, 0, 4, 4, 0, 3,
       3, 6, 4, 4, 1, 0, 3, 4, 5, 4, 3, 1, 0, 7, 1, 6, 4, 0, 3, 4, 4, 0,
       4, 4, 6, 4, 1, 0, 2, 2, 4, 0, 4, 1, 2, 1, 1, 6, 7, 0, 2, 0, 5, 5,
       0, 5, 4, 6, 4, 4, 5, 6, 3, 4, 1, 2, 4, 0, 1, 4, 1, 7, 4, 5, 0, 4,
       0, 0, 0, 1, 4, 5, 1, 0, 1, 2, 4, 1, 4, 0, 1, 5, 0, 7, 4, 1, 0, 1,
       4, 1, 1, 6, 7, 3, 3, 4, 4, 3, 3, 4, 5, 4, 3, 4, 1, 4, 4, 7, 4, 4,
       1, 5, 4, 5, 4, 3, 3, 4, 4, 3, 0, 1, 6, 3, 3, 3, 5, 1, 7, 4, 0, 0,
       3, 3, 6, 0, 3, 0, 3, 1, 0, 2, 6, 3, 2, 5, 1, 2, 5, 1, 0, 5, 6, 4,
       1, 5, 1, 3, 0, 0, 1, 3, 6, 0, 2, 1, 5, 2, 2, 2, 0, 1, 5, 0, 2, 1,
       4, 1, 4, 1, 1, 1, 4, 4, 4, 0, 5, 0, 0, 4, 1, 3, 4, 4, 6, 3, 2, 5,
       3, 4, 2, 5, 4, 0, 5, 4, 2, 4, 0, 5, 0, 1, 5, 3, 1, 4, 1, 4, 0, 3,
       5, 0, 0, 1, 3, 1, 4, 0, 0, 4, 4, 4, 5, 4, 4,

In [ ]:
y_test.shape

(562,)

In [ ]:
print(X_train[0])  # Features for the first sample


[[1.18540001e+02 1.18989998e+02 1.16900002e+02 1.17169998e+02
  1.09322571e+02 1.91900000e+05]
 [1.16970001e+02 1.17360001e+02 1.15209999e+02 1.16629997e+02
  1.08818733e+02 2.89700000e+05]
 [1.16690002e+02 1.16949997e+02 1.15620003e+02 1.15970001e+02
  1.08202942e+02 3.89400000e+05]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [ ]:
print(y_train[0])  # Target for the first sample


1


In [ ]:
X_train

array([[[1.18540001e+02, 1.18989998e+02, 1.16900002e+02, 1.17169998e+02,
         1.09322571e+02, 1.91900000e+05],
        [1.16970001e+02, 1.17360001e+02, 1.15209999e+02, 1.16629997e+02,
         1.08818733e+02, 2.89700000e+05],
        [1.16690002e+02, 1.16949997e+02, 1.15620003e+02, 1.15970001e+02,
         1.08202942e+02, 3.89400000e+05],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00]],

       [[3.30239990e+02, 3.33589996e+02, 3.21739990e+02, 3.30799988e+02,
         3.24019287e+02, 2.35090000e+06],
        [3.31059998e+02, 3.31059998e+02, 3.26820007e+02, 3.29690002e+02,
         3.22932098e+02, 3.39990000e+06],
        [3.27510010e+02, 3.30429993e+02, 3.23459991e+02, 3.241499

In [ ]:
X_test

array([[[2.33874573e+02, 2.38951889e+02, 2.33152924e+02, 2.37757736e+02,
         2.29645874e+02, 8.88481000e+05],
        [2.37070450e+02, 2.39518906e+02, 2.33728516e+02, 2.34037796e+02,
         2.26052811e+02, 7.82906000e+05],
        [2.32903778e+02, 2.36554977e+02, 2.30249146e+02, 2.30807556e+02,
         2.22932800e+02, 5.48011000e+05],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00]],

       [[2.71000004e+01, 2.80949993e+01, 2.68700008e+01, 2.78999996e+01,
         2.75742683e+01, 4.37460000e+06],
        [2.68999996e+01, 2.72000008e+01, 2.55750008e+01, 2.58500004e+01,
         2.55482025e+01, 7.26420000e+06],
        [2.62099991e+01, 2.69899998e+01, 2.58700008e+01, 2.679000

In [ ]:
from sktime.transformations.panel.rocket import Rocket
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sktime.datasets import load_unit_test  # Example dataset
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
rocket = Rocket(num_kernels=10000)
clf = make_pipeline(rocket, LogisticRegression(max_iter=5000, multi_class='multinomial', solver='lbfgs'))


print("Fitting the pipeline to the training dataset...")
clf.fit(X_train, y_train)

Fitting the pipeline to the training dataset...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('rocket', Rocket()),
                ('logisticregression',
                 LogisticRegression(max_iter=5000, multi_class='multinomial'))])

In [ ]:
y_train_probs = clf.predict_proba(X_train)  # Training set probabilities
y_test_probs = clf.predict_proba(X_test)

In [ ]:
y_test_probs

array([[5.32118791e-01, 1.59716765e-07, 3.03071525e-02, ...,
        5.02620379e-09, 3.60293037e-08, 6.41084918e-18],
       [4.81345545e-04, 5.72139915e-04, 4.14437390e-02, ...,
        1.16738466e-06, 1.00402994e-06, 3.00545683e-13],
       [5.84614519e-01, 3.90181269e-01, 4.69311740e-07, ...,
        2.01716060e-03, 1.62604569e-10, 9.83699149e-09],
       ...,
       [6.68501980e-05, 1.73774988e-05, 9.99733924e-01, ...,
        2.80708811e-08, 4.55227646e-08, 2.03821508e-09],
       [9.24732665e-01, 3.99255908e-04, 5.02857850e-06, ...,
        7.48355243e-02, 1.18026593e-06, 2.14573873e-13],
       [5.41395525e-01, 2.63668644e-03, 3.58079519e-03, ...,
        1.04418148e-05, 1.72883047e-06, 2.66614715e-13]])

In [ ]:
y_train_probs

array([[1.92507759e-04, 9.97452563e-01, 1.17964883e-07, ...,
        1.11777656e-07, 1.61578938e-11, 1.73775490e-10],
       [1.87794140e-05, 9.97021690e-01, 4.21259412e-07, ...,
        6.45841649e-04, 6.39794602e-09, 4.69337916e-08],
       [9.99336427e-01, 3.50925005e-06, 5.15316334e-06, ...,
        3.10449020e-04, 6.09923259e-07, 5.56283957e-14],
       ...,
       [9.98259476e-01, 1.74277217e-06, 1.73775675e-03, ...,
        7.18656600e-10, 6.26326362e-13, 9.04535052e-20],
       [9.96181238e-01, 2.87388989e-03, 8.42270077e-06, ...,
        8.06790710e-05, 7.84747684e-06, 4.71292586e-10],
       [2.52620494e-06, 2.25758785e-12, 1.74255029e-04, ...,
        3.70267465e-05, 3.97739439e-15, 7.93411359e-08]])

In [ ]:
from sklearn.metrics import accuracy_score

# Training accuracy
y_train_pred = y_train_probs.argmax(axis=1)  # Predicted classes for training data
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.2f}")

# Testing accuracy
y_test_pred = y_test_probs.argmax(axis=1)  # Predicted classes for test data
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")


Training Accuracy: 1.00
Test Accuracy: 0.61


In [ ]:
from xgboost import XGBClassifier
rocket = Rocket(num_kernels=10000)
clf_xgb = make_pipeline(
    rocket,
    XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', n_estimators=100)
)
clf_xgb.fit(X_train, y_train)


In [ ]:
y_train_probs = clf_xgb.predict_proba(X_train) 
y_test_probs = clf_xgb.predict_proba(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

# Training accuracy
y_train_pred = y_train_probs.argmax(axis=1)  
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.2f}")

# Testing accuracy
y_test_pred = y_test_probs.argmax(axis=1)  
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")